## Sanity check for Aurora

I think I caused confusion (or I was confused) as to the zero point used in the original calculation. So let's do it all over again to ensure that 

* Everything is consistent with $cz=1307 \mathrm{km~s}^{-1}$, which is the zero point for M87 that Aurora wants to use consistently throughout the paper

* The Gas Mass is calculated using a Luminosity Distance of 16.1 Mpc

To ensure everything is consistent, I'll do everything from scratch: 

### 0. Solving a Mystery

The WTF with the redshift was my fault. In my notes, I had written down that I was using a $z=0.004283$, which is the first redshift listed in NED. I had confused myself, and thought that my zero point in the CASA imaging script was indeed $z=0.004283$, _but it wasn't_. My bad! 

The whole time, I was actually using $z=0.004343$ in the actual CASA script. I didn't notice this because it was in terms of the redshifted CO(2-1) frequency, which I originally assumed was set using the NED redshift. 

Long story short, I was using 1302 km/s, you're wanting 1307 km/s, so we don't expect a 23 km/s offset between my original fit and this new one. The entire time, I was using 1302 km/s, or a z of 0.00434. 

Below, everything is recalculated assuming $cz = 1307$ km/s

In [85]:
from astropy import units as u
from astropy import constants as const

from astropy.cosmology import FlatLambdaCDM, z_at_value

In [86]:
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

### 1. Re-image the Measurement set with 5 km/s channel widths and a zero point of 1307 km/s

In [87]:
lightspeed = const.c.to(u.km / u.s)
lightspeed

<Quantity 299792.458 km / s>

The redshift that Aurora wants to consistenly use throughout the paper is  $cz = 1307~\mathrm{km~s}^{-1}$, i.e. a redshift of: 

In [88]:
cz = 1307 * u.km / u.s

z = cz / lightspeed
print("Use this redshift: z=",  z)

Use this redshift: z= 0.004359682724239848


In [89]:
lightspeed * 0.004283

<Quantity 1284.011097614 km / s>

I therefore need to use this rest frequency when imaging the measurement set

In [90]:
co21_restfreq = 230.53800 * u.GHz # This is the v=0 rest frequency of CO(2-1), straight from the Splatalogue
co21_redshifted = co21_restfreq / (1+z)
co21_redshifted

<Quantity 229.53729024116677 GHz>

This is the frequency that I put as ```restfreq``` (yes, it's misleading - it's just the zero point for the cube in `velocity` mode) in the ```CLEAN``` step. 

Here's the imaging setup. We do 5 km/s channels, ```natural``` weighting, PB correction, and a zero point of $cz=1307 \mathrm{km~s}^{-1}$

<img src="1.png">
<img src="2.png">


### 2. Fit a gaussian to the line in this newly imaged cube

<img src="fitpreview.png">
<img src="finalfit.png">

## Final values to use for paper: 
For consistency, let's go with the above fit to the 5 km/s cube. The results of this fit varies depending on the binning of the cube, how it was imaged, etc., but those variations are consistent within the below errorbars

* Line center:  $-129 \pm 3~\mathrm{km s}^{-1}$, where the zeropoint is 1307 km/s
* Line FWHM: $64 \pm 7 ~\mathrm{km s}^{-1}$
* Line sigma: $\sigma = 27 \pm 3 \mathrm{km~s}^{-1}$
* Emission integral: $0.557 \pm 0.057~\mathrm{Jy~km}^{-1}$

### Compute the gas mass using a Luminosity Distance of 16.1 Mpc

In [91]:
def gasmass(gaussian_area, gaussian_area_err, ldist, redshift):
    '''
    Compute a gas mass for M87. 
    The gaussian area must be in mJy * km/s (becuase I explicitly convert to Jy * km/s below)
    '''
    
    emission_integral_units_mJy = u.mJy * u.km / u.s
    emission_integral_units_Jy = u.Jy * u.km / u.s

    xco_unit = u.cm**(-2) * (u.K * u.km * u.s**(-1))**(-1)
    
    emission_integral = (gaussian_area * emission_integral_units_mJy).to(emission_integral_units_Jy)
    emission_integral_err = (gaussian_area_err * emission_integral_units_mJy).to(emission_integral_units_Jy)
    
    ldist = ldist * u.Mpc
    redshift = redshift
    xco = 2.0e20 * xco_unit
    ratio = 3.2  # CO(2-1) to CO(1-0) flux density ratio 
    xco_mw = 2.0e20 * xco_unit # CO X-factor. We use the MWs for lack of a more informed choice.
    
    rawmass = 1.05e4 * (xco / xco_mw) * (1 + redshift)**(-1) * (emission_integral / emission_integral_units_Jy) * (ldist/u.Mpc)**2 * (1.0/ratio)
    rawmass_err = 1.05e4 * (xco / xco_mw) * (1 + redshift)**(-1) * (emission_integral_err / emission_integral_units_Jy) * (ldist/u.Mpc)**2 * (1.0/ratio)
    
    
    
    finalmass = rawmass * u.M_sun
    finalmass_err = rawmass_err * u.M_sun
       
    return finalmass, finalmass_err, emission_integral

In [93]:
h2mass = gasmass(557, 57, 16.1, 0.0043596)
h2mass

(<Quantity 471690.3951159527 solMass>,
 <Quantity 48269.93271384076 solMass>,
 <Quantity 0.557 Jy km / s>)

### So, yeah, this is a factor of two lower than the value we're reporting in the current draft of the paper.

$M_{\mathrm{H_2}} = (4.7 \pm 0.4) \times 10^5$ $M_\odot$

This is largely driven by the differing $L_D$ (16.1 vs. 18.4 Mpc, which is what I was using previously), but also by the fact that the best-fit gaussian has a slightly larger area because of velocity  

To give you an idea of how much this is driven by the different luminosity difference, see below: This is the emission integral from the fit to the 10km/s cube in the original jupyter notebook I sent you. The emission integral changes by about 50\%, so, certainly not insignificant. But the errors on the 10 km/s fit are far smaller. I think it's still totally fine to use that one. Ultimately, given the rather large uncertainties involved (X_CO, etc.), H2 gas masses are very order-of-magnitude. 

In [101]:
orig_gasmass = gasmass(817, 118, 16.1, 0.0043596)
orig_gasmass

(<Quantity 691869.0355650509 solMass>,
 <Quantity 99927.2291268984 solMass>,
 <Quantity 0.8170000000000001 Jy km / s>)